# Notebook to download all reference papers and extract txt files from them
I've made a list of references - "reference_list.txt" and reading information from there. 

In [1]:
reference_txt = open("reference_list.txt", 'r').readlines()

In [39]:
# Get titles from 
def parse_reference(ref):
    asd = ",".join(ref.split(",")[:-1])
    asd = asd.split(". ")[-1]
    # asd = list(filter(lambda x: len(x) > 10, asd))
    return asd

list_of_paper_names = []

for r in reference_txt:
    list_of_paper_names.append(parse_reference(r))
    

In [40]:
list_of_paper_names

['Alloying behavior of iron, gold and silver in AlCoCrCuNi-based equimolar high-entropy alloys',
 'FCC and BCC equivalents in as-cast solid solutions of AlxCo yCrzCu0.5FevNiw high-entropy alloys',
 'Alloying behavior in multi-component AlCoCrCuFe and NiCoCrCuFe high entropy alloys',
 'Effects of Mn, Ti and V on the microstructure and properties of AlCrFeCoNiCu high entropy alloy',
 'Microstructure and compressive properties of multicomponent Alx(TiVCrMnFeCoNiCu)100−x high-entropy alloys',
 'Microstructures and compressive properties of multicomponent AlCoCrCuFeNiMox alloys',
 'Phase transformation induced by lattice distortion in multiprincipal component CoCrFeNiCuxAl1−x solid-solution alloys',
 'Morphology, structure and composition of precipitates in Al0.3CoCrCu0.5FeNi high-entropy alloy',
 'Nanostructured high-entropy alloys with multiple principal elements: Novel alloy design concepts and outcomes',
 'Effect of the substitution of Co by Mn in Al-Cr-Cu-Fe-Co-Ni high-entropy alloys',

In [43]:
from scidownl import scihub_download
import os
import re

try:
    last_pdf_done = max(list(map(lambda x: int(re.search("reference_([0-9]+)", x).groups()[0]), os.listdir("references/"))))
except: 
    last_pdf_done = 0

print(f"Starting from reference_{last_pdf_done+1}")
# Needs to be re-run if stuck
for i in range(len(list_of_paper_names)):
    if i + 1 <= last_pdf_done or i+1 in [41]: # 41 gets stuck, idk
        continue
    print(i+1)
    scihub_download(list_of_paper_names[i], paper_type="title", out=f"references/reference_{i+1}.pdf")

[INFO] | 2023/12/06 12:33:19 | Choose scihub url [0]: https://sci-hub.ru
[INFO] | 2023/12/06 12:33:19 | <- Request: scihub_url=https://sci-hub.ru, source=TitleSource[type=title, id=Effect of iron content on wear behavior of AlCoCrFexMo0.5Ni high-entropy alloys], proxies={}


Starting from reference_42
42


[INFO] | 2023/12/06 12:33:19 | -> Response: status_code=200, content_length=5895
[WARNING] | 2023/12/06 12:33:19 | Error occurs, task status: extracting_failed, error: No pdf tag was found in the given content with the selector: #pdf
[INFO] | 2023/12/06 12:33:19 | Choose scihub url [1]: http://sci-hub.ru
[INFO] | 2023/12/06 12:33:19 | <- Request: scihub_url=http://sci-hub.ru, source=TitleSource[type=title, id=Effect of iron content on wear behavior of AlCoCrFexMo0.5Ni high-entropy alloys], proxies={}
[INFO] | 2023/12/06 12:33:20 | -> Response: status_code=200, content_length=5895
[WARNING] | 2023/12/06 12:33:20 | Error occurs, task status: extracting_failed, error: No pdf tag was found in the given content with the selector: #pdf
[INFO] | 2023/12/06 12:33:20 | Choose scihub url [2]: https://sci-hub.st
[INFO] | 2023/12/06 12:33:20 | <- Request: scihub_url=https://sci-hub.st, source=TitleSource[type=title, id=Effect of iron content on wear behavior of AlCoCrFexMo0.5Ni high-entropy alloys

43


[INFO] | 2023/12/06 12:33:27 | -> Response: status_code=200, content_length=7654
[INFO] | 2023/12/06 12:33:27 | * Extracted information: {'url': 'https://moscow.sci-hub.ru/312/c86e63e8e34128d9f605ca5a208dc47a/zhu2010.pdf', 'title': 'Microstructures and compressive properties of multicomponent AlCoCrFeNiMox alloys. Materials Science and Engineering  A, 527(26), 6975–6979'}


...

[INFO] | 2023/12/06 12:39:17 | -> Response: status_code=200, content_length=7593
[INFO] | 2023/12/06 12:39:17 | * Extracted information: {'url': 'https://dacemirror.sci-hub.ru/journal-article/f72100ba8026480ec481dce17a8c5d27/zhang2004.pdf', 'title': 'Bulk Glass Formation of 12 mm Rod in La–Cu–Ni–Al Alloys. Materials Science and Engineering  A, 375-377, 436–439'}


100% [==================================================] 747/747


[INFO] | 2023/12/06 12:39:17 | ↓ Successfully download the url to: references/reference_120.pdf
[INFO] | 2023/12/06 12:39:17 | Choose scihub url [0]: https://sci-hub.ru
[INFO] | 2023/12/06 12:39:17 | <- Request: scihub_url=https://sci-hub.ru, source=TitleSource[type=title, id=Phase equilibria in the quaternary molybdenum-ruthenium-rhodium-palladium system], proxies={}


121


[INFO] | 2023/12/06 12:39:18 | -> Response: status_code=200, content_length=5888
[WARNING] | 2023/12/06 12:39:18 | Error occurs, task status: extracting_failed, error: No pdf tag was found in the given content with the selector: #pdf
[INFO] | 2023/12/06 12:39:18 | Choose scihub url [1]: http://sci-hub.ru
[INFO] | 2023/12/06 12:39:18 | <- Request: scihub_url=http://sci-hub.ru, source=TitleSource[type=title, id=Phase equilibria in the quaternary molybdenum-ruthenium-rhodium-palladium system], proxies={}
[INFO] | 2023/12/06 12:39:19 | -> Response: status_code=200, content_length=5888
[WARNING] | 2023/12/06 12:39:19 | Error occurs, task status: extracting_failed, error: No pdf tag was found in the given content with the selector: #pdf
[INFO] | 2023/12/06 12:39:19 | Choose scihub url [2]: https://sci-hub.st
[INFO] | 2023/12/06 12:39:19 | <- Request: scihub_url=https://sci-hub.st, source=TitleSource[type=title, id=Phase equilibria in the quaternary molybdenum-ruthenium-rhodium-palladium syst

Lets see which ones are missing

In [49]:
not_dones = sorted(list(set(list(range(len(list_of_paper_names)))) - set(list(map(lambda x: int(re.search("reference_([0-9]+)", x).groups()[0]), os.listdir("references/"))))))
print(f"Missing references: {len(not_dones)}/{len(list_of_paper_names)} (can't do anything about it..)")
", ".join(list(map(str, not_dones)))

Missing references: 41/121 (can't do anything about it..)


'0, 2, 8, 10, 13, 15, 16, 18, 19, 20, 21, 22, 23, 32, 34, 39, 40, 41, 42, 49, 51, 61, 62, 65, 66, 67, 71, 74, 75, 77, 78, 80, 83, 84, 85, 94, 97, 98, 108, 110, 115'

# Convert to txt files (along with some processing to only keep "content")

In [54]:
from pdfminer.high_level import extract_text
import glob

for file in glob.glob("references/*"):
    print(file)
    raw = extract_text(file)
    break

references/reference_86.pdf


In [64]:
raw.split("\n")

['Intermetallics 15 (2007) 357e362',
 '',
 'www.elsevier.com/locate/intermet',
 '',
 'Novel microstructure and properties of multicomponent',
 'CoCrCuFeNiTix alloys',
 '',
 'X.F. Wang, Y. Zhang*, Y. Qiao, G.L. Chen',
 '',
 'State Key Laboratory for Advanced Metals and Materials, University of Science and Technology Beijing,',
 'XueYuan Road 30#, Haidian District, Beijing 100083, China',
 '',
 'Received 10 May 2006; received in revised form 7 July 2006; accepted 4 August 2006',
 'Available online 5 October 2006',
 '',
 'Abstract',
 '',
 'CoCrCuFeNiTix (x values in molar ratio, x ¼ 0, 0.5, 0.8 and 1.0) were prepared by arc melting of the pure elements and suction casting under an',
 'argon atmosphere. Both CoCrCuFeNi and CoCrCuFeNiTi0.5 alloys form a single FCC solid solution. While the alloys of CoCrCuFeNiTi0.8 and',
 'CoCrCuFeNiTi are basically composed of primary FCC solid solution and eutectic mixture of FCC phase and Laves phase of Fe2Ti type. The yield',
 'strength of the alloys in

In [62]:
import re

s = re.findall("abstract", raw, re.IGNORECASE)
assert len(s) == 1, f"There's more than 1 match for 'abstract' : {s}"

raw.split(s[0])

['Intermetallics 15 (2007) 357e362\n\nwww.elsevier.com/locate/intermet\n\nNovel microstructure and properties of multicomponent\nCoCrCuFeNiTix alloys\n\nX.F. Wang, Y. Zhang*, Y. Qiao, G.L. Chen\n\nState Key Laboratory for Advanced Metals and Materials, University of Science and Technology Beijing,\nXueYuan Road 30#, Haidian District, Beijing 100083, China\n\nReceived 10 May 2006; received in revised form 7 July 2006; accepted 4 August 2006\nAvailable online 5 October 2006\n\n',
 '\n\nCoCrCuFeNiTix (x values in molar ratio, x ¼ 0, 0.5, 0.8 and 1.0) were prepared by arc melting of the pure elements and suction casting under an\nargon atmosphere. Both CoCrCuFeNi and CoCrCuFeNiTi0.5 alloys form a single FCC solid solution. While the alloys of CoCrCuFeNiTi0.8 and\nCoCrCuFeNiTi are basically composed of primary FCC solid solution and eutectic mixture of FCC phase and Laves phase of Fe2Ti type. The yield\nstrength of the alloys increases from 230 MPa to 1272 MPa with the increase of Ti additi

In [58]:
raw

'Intermetallics 15 (2007) 357e362\n\nwww.elsevier.com/locate/intermet\n\nNovel microstructure and properties of multicomponent\nCoCrCuFeNiTix alloys\n\nX.F. Wang, Y. Zhang*, Y. Qiao, G.L. Chen\n\nState Key Laboratory for Advanced Metals and Materials, University of Science and Technology Beijing,\nXueYuan Road 30#, Haidian District, Beijing 100083, China\n\nReceived 10 May 2006; received in revised form 7 July 2006; accepted 4 August 2006\nAvailable online 5 October 2006\n\nAbstract\n\nCoCrCuFeNiTix (x values in molar ratio, x ¼ 0, 0.5, 0.8 and 1.0) were prepared by arc melting of the pure elements and suction casting under an\nargon atmosphere. Both CoCrCuFeNi and CoCrCuFeNiTi0.5 alloys form a single FCC solid solution. While the alloys of CoCrCuFeNiTi0.8 and\nCoCrCuFeNiTi are basically composed of primary FCC solid solution and eutectic mixture of FCC phase and Laves phase of Fe2Ti type. The yield\nstrength of the alloys increases from 230 MPa to 1272 MPa with the increase of Ti addi